In [88]:
import akshare as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import calendar

# 获取板块股票
def get_industry_stocks(industry='白酒') -> pd.DataFrame:
    stocks = ak.stock_board_industry_cons_ths(symbol=industry)

    return pd.DataFrame({
        "code": stocks["代码"],
        "name": stocks["名称"],
        "tradable": stocks["流通股"],
        "market_value": stocks["流通市值"]
    })


# 获取股票价格
def get_stock_price(symbol: str, start_date: datetime, end_date: datetime, adjust: str) -> pd.DataFrame:
    stock_hist = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=start_date.strftime("%Y%m%d"), 
        end_date=end_date.strftime("%Y%m%d"), adjust=adjust)
    
    return pd.DataFrame({
        'open': list(stock_hist["开盘"]),
        'close': list(stock_hist["收盘"]),
        'high': list(stock_hist["最高"]),
        'low': list(stock_hist["最低"]),
    }, index=pd.to_datetime(stock_hist["日期"]))

def get_stock_eps(symbol: str) -> pd.DataFrame:
    market = 'sz'
    if (symbol.startswith('00') != True):
        market ='sh'
    stock = "{}{}".format(market, symbol)
    profit_y = ak.stock_profit_sheet_by_yearly_em(symbol=stock)

    eps = pd.Series(list(profit_y["BASIC_EPS"]), index=pd.to_datetime(profit_y["REPORT_DATE"]))

    df_eps = pd.DataFrame({
        "eps": eps
        })
    df_eps.fillna(0, inplace=True)

    return df_eps

def get_stock_dividents(symbol: str) -> pd.DataFrame:
    df_dividents = pd.DataFrame()
    dividents_dict = {}

    dividents = ak.stock_dividents_cninfo(symbol=symbol)
    
    # print(dividents["报告时间"])
    dateIndex = dividents["报告时间"].str[:4].apply(lambda x: datetime.date(int(x), 12, 31) if x is not None else None)
    
    df_dividents = pd.DataFrame({
        "sg": list(dividents["送股比例"]),
        "zz": list(dividents["转增比例"]),
        "px": list(dividents["派息比例"]),
        "desc": list(dividents["实施方案分红说明"]),
        "type": list(dividents["分红类型"])
    }, index = pd.to_datetime(dateIndex))

    df_dividents.index.set_names("REPORT_DATE", inplace=True)
    df_dividents.fillna(0, inplace=True)  

    return df_dividents[df_dividents["type"] == "年度分红"]

In [87]:
print(get_stock_dividents("600519"))

0       None
1     2021年报
2     2020年报
3     2019年报
4     2018年报
5     2017年报
6     2016年报
7     2015年报
8     2014年报
9     2013年报
10    2012年报
11    2011年报
12    2010年报
13    2009年报
14    2008年报
15    2007年报
16    2006年报
17      None
18      None
19    2005年报
20    2004年报
21    2003年报
22    2002年报
23    2001年报
Name: 报告时间, dtype: object
              sg   zz      px              desc  type
REPORT_DATE                                          
2021-12-31   0.0  0.0  216.75    10派216.75元(含税)  年度分红
2020-12-31   0.0  0.0  192.93    10派192.93元(含税)  年度分红
2019-12-31   0.0  0.0  170.25    10派170.25元(含税)  年度分红
2018-12-31   0.0  0.0  145.39    10派145.39元(含税)  年度分红
2017-12-31   0.0  0.0  109.99    10派109.99元(含税)  年度分红
2016-12-31   0.0  0.0   67.87     10派67.87元(含税)  年度分红
2015-12-31   0.0  0.0   61.71     10派61.71元(含税)  年度分红
2014-12-31   1.0  0.0   43.74  10送1股派43.74元(含税)  年度分红
2013-12-31   1.0  0.0   43.74  10送1股派43.74元(含税)  年度分红
2012-12-31   0.0  0.0   64.19     10派64.19元(含税)  年度分红
2011-12-31   0

In [95]:
stocks = get_industry_stocks("银行")


for i in tqdm(range(len(stocks))):
    code = stocks.loc[i, "code"]
    name = stocks.loc[i, "name"]
    print(code)

    df_div = get_stock_dividents(code)
    df_eps = get_stock_eps(code)
    price = get_stock_price(code, start_date=datetime.date(2000,1,1), end_date=datetime.date(2021,12,31), adjust="")

    df = pd.concat([price.resample('Y')["low"].min(),price.resample('Y')["high"].max(), df_eps, df_div], axis=1, join='outer')
    df["2010":].to_excel(f"./银行/{code}-{name}.xlsx")

  0%|          | 0/42 [00:00<?, ?it/s]       

002966


  2%|▏         | 1/42 [00:01<01:16,  1.87s/it]

601009


  5%|▍         | 2/42 [00:04<01:33,  2.35s/it]

...

601128


 83%|████████▎ | 35/42 [01:31<00:21,  3.06s/it]

001227


 83%|████████▎ | 35/42 [01:33<00:18,  2.68s/it]


KeyError: '开盘'

In [92]:
ak.stock_board_industry_name_ths().to_csv("行业.csv")